<a href="https://colab.research.google.com/github/aniishap/ds2002/blob/main/Data_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL Data Processor Project

Target DB Schema: Sakila Database



### Import Necessary Libraries

In [2]:
!pip install PyMySQL
!pip install mysql.connector
!pip install sqlalchemy
!pip install mysql-connector-python
!pip install pyMysql
!pip install pymongo





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 657.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 25.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp310-cp310-linux_x86_64.whl size=247953 sha256=2c5182e40580e616d89dc2758a305fc397bacd34b02a8f9ce753bb244a72f80c
  Stored in directory: /root/.cache/pip/wheels/76/48/9b/da67ff1a18fe8e9d428f9b1a177716d4a7d363d2bbe83bf6cf
Successfully built mysql.connector
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.9/676.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 11.8 MB/s eta 0:00:00


In [3]:
import os
import datetime
import pymongo
import pprint
import pandas as pd
import numpy
import pandas as pd
from sqlalchemy import create_engine


### Declare & Assign Connection Variables to query MySQL using Python


> Add blockquote



In [4]:


#Set the host name here
#If you are running against a database locally it would be local host or 127.0.0.1
host_name = "database.ds2002.org"
#host_ip = "127.0.0.1"
port = "3306"

user_id = "euv7bw" # my computing id
pwd = "euv7bw!"  #
db_name = "sakila" # db i want to use


### Verify that the connection is succesful

In [5]:

import pymysql

conn = pymysql.connect(
    host=host_name,
    user=user_id,
    password=pwd
)

print("Connected successfully")
conn.close()

Connected successfully


### Define Functions for Getting Data From and Setting Data Into Databases (MySQL, phpMyAdmin)


In [6]:

def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()

    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()

    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")

    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')

    connection.close()



### Create & Populate New Dimension Tables: Extracting Data from Sakila Database

SELECT and extract tables chosen for database idea. From the sakila database, I decided to choose these 5 tables: **film, film_category, category, film_actor, and actor**. I decided to choose these 5 ables because they related to each other through primary and foreign keys. The implication of this database is that these tables can keep track and manage various film types and their inventories. Overall, it keeps track of movie rentals and management.

In [7]:

sql_film = "SELECT * FROM sakila.film;"
df_film = get_dataframe(user_id, pwd, host_name,db_name, sql_film)
df_film.head(5)




,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,None,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,None,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,None,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [8]:
sql_filmcategory = "SELECT * FROM sakila.film_category;"
df_filmcategory = get_dataframe(user_id, pwd, host_name,db_name, sql_filmcategory)
df_filmcategory.head(5)

,film_id,category_id,last_update
0,1,6,2006-02-15 05:07:09
1,2,11,2006-02-15 05:07:09
2,3,6,2006-02-15 05:07:09
3,4,11,2006-02-15 05:07:09
4,5,8,2006-02-15 05:07:09


In [9]:
sql_category = "SELECT * FROM sakila.category;"
df_category = get_dataframe(user_id, pwd, host_name,db_name, sql_category)
df_category.head(5)

,category_id,name,last_update
0,1,Action,2006-02-15 04:46:27
1,2,Animation,2006-02-15 04:46:27
2,3,Children,2006-02-15 04:46:27
3,4,Classics,2006-02-15 04:46:27
4,5,Comedy,2006-02-15 04:46:27


In [10]:
sql_filmactor = "SELECT * FROM sakila.film_actor;"
df_filmactor = get_dataframe(user_id, pwd, host_name,db_name, sql_filmactor)
df_filmactor.head(5)

,actor_id,film_id,last_update
0,1,1,2006-02-15 05:05:03
1,1,23,2006-02-15 05:05:03
2,1,25,2006-02-15 05:05:03
3,1,106,2006-02-15 05:05:03
4,1,140,2006-02-15 05:05:03


In [11]:
sql_actor = "SELECT * FROM sakila.actor;"
df_actor = get_dataframe(user_id, pwd, host_name,db_name, sql_actor)
df_actor.head(5)

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


### Perform Any Necessary Transformations to the DataFrames

Columns were dropped from the 5 dataframes to ensure that data is concise and relevant to the selected database design of tracking movie rental types and management.

In [12]:
drop_cols = ['last_update', 'original_language_id', 'special_features', 'description']
df_film.drop(drop_cols, axis=1, inplace=True)
df_film.head(2)

,film_id,title,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating
0,1,ACADEMY DINOSAUR,2006,1,6,0.99,86,20.99,PG
1,2,ACE GOLDFINGER,2006,1,3,4.99,48,12.99,G


In [13]:
drop_cols = ['last_update']
df_filmactor.drop(drop_cols, axis=1, inplace=True)
df_filmactor.head(2)

,actor_id,film_id
0,1,1
1,1,23


In [14]:
drop_cols = ['last_update']
df_filmcategory.drop(drop_cols, axis=1, inplace=True)
df_filmcategory.head(2)

,film_id,category_id
0,1,6
1,2,11


In [15]:
drop_cols = ['last_update']
df_actor.drop(drop_cols, axis=1, inplace=True)
df_actor.head(2)

,actor_id,first_name,last_name
0,1,PENELOPE,GUINESS
1,2,NICK,WAHLBERG


In [16]:
drop_cols = ['last_update']
df_category.drop(drop_cols, axis=1, inplace=True)
df_category.head(2)


,category_id,name
0,1,Action
1,2,Animation


## Load the Transformed DataFrames into the New Data Warehouse by Creating New Tables using MongoDB


In [23]:
import mysql.connector

# Define connection parameters
host = 'database.ds2002.org'  # Replace with your MySQL host address
user = 'euv7bw'     # Replace with your MySQL username
password = 'euv7bw!' # Replace with your MySQL password
database = 'euv7bw' # Replace with your MySQL database name

# Connect to MySQL
try:
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")
except mysql.connector.Error as err:
    print("Error connecting to MySQL database:", err)



Connected to MySQL database


In [24]:
#Show the current tables
cursor = conn.cursor()
cursor.execute("SHOW TABLES")


tables = cursor.fetchall()  # Fetch all tables
for (table_name,) in tables:
    print(table_name)

# Close the cursor and connection
cursor.close()
conn.close()

film
testtable


### Create film Table

In [25]:
try:
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")
except mysql.connector.Error as err:
    print("Error connecting to MySQL database:", err)

if conn.is_connected():
    print("Successfully connected to the database")

cursor = conn.cursor()

create_film_query = """
CREATE TABLE film (
  film_id INT AUTO_INCREMENT PRIMARY KEY,
  title VARCHAR(128) NOT NULL,
  release_year YEAR,
  language_id INT,
  rental_duration INT,
  rental_rate DECIMAL(4,2),
  length INT,
  replacement_cost DECIMAL(5,2),
  rating ENUM('G', 'PG', 'PG-13', 'R', 'NC-17'))
"""

try:
    # Execute the CREATE TABLE query
    cursor.execute(create_film_query)
    print("Table 'example_table' created successfully.")
except mysql.connector.Error as err:
    print(f"Failed to create table: {err}")

#Let's prove it..you should see two tables now..your new one and my old testtable
cursor.execute("SHOW TABLES")


tables = cursor.fetchall()  # Fetch all tables
for (table_name,) in tables:
    print(table_name)


# Close the cursor and connection
cursor.close()
conn.close()


Connected to MySQL database
Successfully connected to the database
Failed to create table: 1050 (42S01): Table 'film' already exists
film
testtable


In [21]:
try:
    # Establish a connection to the MySQL database
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")
except mysql.connector.Error as err:
    print("Error connecting to MySQL database:", err)
    exit(1)

if conn.is_connected():
    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    try:
        # Execute a SELECT query to retrieve data from the film table
        cursor.execute("SELECT * FROM film")

        # Fetch all rows from the result set
        films = cursor.fetchall()

        # Display the fetched data
        for film in films:
            print(film)  # You can customize the printing format as per your requirements

    except mysql.connector.Error as err:
        print("Error executing SELECT query:", err)
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

Connected to MySQL database


## Create actor table

In [27]:
try:
    # Establish a connection to the MySQL database
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")
except mysql.connector.Error as err:
    print("Error connecting to MySQL database:", err)
    exit(1)

if conn.is_connected():
    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    # Define the CREATE TABLE statement for the actor table
    create_actor_table_query = """
    CREATE TABLE actor (
      actor_id INT AUTO_INCREMENT PRIMARY KEY,
      first_name VARCHAR(45) NOT NULL,
      last_name VARCHAR(45) NOT NULL
    )
    """

    try:
        # Execute the CREATE TABLE query
        cursor.execute(create_actor_table_query)
        print("Table 'actor' created successfully.")
    except mysql.connector.Error as err:
        print(f"Failed to create table: {err}")
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

Connected to MySQL database
Table 'actor' created successfully.


## Create film_actor table

In [28]:
try:
    # Establish a connection to the MySQL database
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")
except mysql.connector.Error as err:
    print("Error connecting to MySQL database:", err)
    exit(1)

if conn.is_connected():
    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    # Define the CREATE TABLE statement for the film_actor table
    create_film_actor_table_query = """
    CREATE TABLE film_actor (
      actor_id INT,
      film_id INT,
      FOREIGN KEY (actor_id) REFERENCES actor(actor_id),
      FOREIGN KEY (film_id) REFERENCES film(film_id),
      PRIMARY KEY (actor_id, film_id)
    )
    """

    try:
        # Execute the CREATE TABLE query
        cursor.execute(create_film_actor_table_query)
        print("Table 'film_actor' created successfully.")
    except mysql.connector.Error as err:
        print(f"Failed to create table: {err}")
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

Connected to MySQL database
Table 'film_actor' created successfully.


## Create category table

In [29]:
try:
    # Establish a connection to the MySQL database
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")
except mysql.connector.Error as err:
    print("Error connecting to MySQL database:", err)
    exit(1)

if conn.is_connected():
    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    # Define the CREATE TABLE statement for the category table
    create_category_table_query = """
    CREATE TABLE category (
      category_id INT AUTO_INCREMENT PRIMARY KEY,
      name VARCHAR(45) NOT NULL
    )
    """

    try:
        # Execute the CREATE TABLE query
        cursor.execute(create_category_table_query)
        print("Table 'category' created successfully.")
    except mysql.connector.Error as err:
        print(f"Failed to create table: {err}")
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

Connected to MySQL database
Table 'category' created successfully.


## Create film_category table

In [30]:
try:
    # Establish a connection to the MySQL database
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print("Connected to MySQL database")
except mysql.connector.Error as err:
    print("Error connecting to MySQL database:", err)
    exit(1)

if conn.is_connected():
    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    # Define the CREATE TABLE statement for the film_category table
    create_film_category_table_query = """
    CREATE TABLE film_category (
      film_id INT,
      category_id INT,
      FOREIGN KEY (film_id) REFERENCES film(film_id),
      FOREIGN KEY (category_id) REFERENCES category(category_id),
      PRIMARY KEY (film_id, category_id)
    )
    """

    try:
        # Execute the CREATE TABLE query
        cursor.execute(create_film_category_table_query)
        print("Table 'film_category' created successfully.")
    except mysql.connector.Error as err:
        print(f"Failed to create table: {err}")
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

Connected to MySQL database
Table 'film_category' created successfully.
